In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import time
from tqdm import tqdm
import pandas as pd

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [ ]:
print(os.listdir("../input/data/"))

In [ ]:
from glob import glob
rawImagePaths = glob('../input/data/images*/images/*.png')
USE_42GB = True
if USE_42GB:
    df = pd.read_csv("/kaggle/input/data/Data_Entry_2017.csv")
else:
    df = pd.read_csv("/kaggle/input/sample/sample_labels.csv")
print('Number of Image Paths: ', len(rawImagePaths))

In [ ]:
df.head()

In [ ]:
x = df.loc[:,'Finding Labels']

In [ ]:
label_counts = df['Finding Labels'].unique()
label_counts

In [ ]:
from itertools import chain
# df['Finding Labels'] = df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
all_labels = np.unique(list(chain(*df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
# df['Finding Labels']
all_labels = [x for x in all_labels if len(x)>0]

for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        df[c_label] = df['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)
df.sample(3)

In [ ]:
if USE_42GB:
    rakeshSamplingConstant = 1200
    df_42GB = pd.DataFrame()
    for label in all_labels:
        df_temp = df.loc[df['Finding Labels'].str.contains(label)]
        if len(df_temp) >= rakeshSamplingConstant:
            df_temp = df_temp.sample(rakeshSamplingConstant)
            print("Obtained", len(df_temp), "\trandom samples for label: \t", label)
        else:
            #gotta catchem all #pokemon
            print("Obtained", len(df_temp), "\tsamples for label: \t\t", label)

        df_42GB = pd.concat([df_42GB, df_temp],ignore_index=True)
    df = df_42GB
    print("Total new samples: ", len(df))


In [ ]:
#link df to image, takes about 30 seconds
mappedImagePaths = []
if USE_42GB:
    count = 0
    start = time.time()
    for imageName in df["Image Index"]:
        for path in rawImagePaths:
            if imageName in path:
                count+=1
                mappedImagePaths.append(path);
                #print (path, "\t", imageName)
                break
    end = time.time()
    print(end-start)
    print (count)

In [ ]:
#verify propper mapping of panda df and the mapped image paths
if USE_42GB:
    for path, imageName in zip (mappedImagePaths, df["Image Index"]):
        print (path, imageName)
        if (imageName not in path):
            print("ERROR")

In [ ]:
df['disease_vec'] = df.apply(lambda x: [x[all_labels].values], 1).map(lambda x: x[0])
y=df['disease_vec']
y[:10]

In [ ]:
train_data_ = []
path = "/kaggle/input/sample/sample/images"
IMG_SIZE =100

In [ ]:
def create_training_data_42GB():
    i=0
    for imgPath in tqdm(mappedImagePaths):
        try:
            #print (imgPath)
            img_array1 = cv2.imread(imgPath,cv2.IMREAD_GRAYSCALE)  
            new_array1 = cv2.resize(img_array1, (IMG_SIZE, IMG_SIZE))  
            train_data_.append([new_array1,y[i]])  
            i = i+1
        except Exception as e: 
            pass

In [ ]:
def create_training_data1():
    i=0
    for img in tqdm(os.listdir(path)):
        try:
            img_array1 = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
            new_array1 = cv2.resize(img_array1, (IMG_SIZE, IMG_SIZE))  
            train_data_.append([new_array1,y[i]])  
            i = i+1
        except Exception as e: 
            pass

In [ ]:
if (USE_42GB):
    create_training_data_42GB()
else:
    create_training_data1()

print(len(train_data_))

In [ ]:
import random
random.shuffle(train_data_) 
print(train_data_[0])

In [ ]:
import pickle
filename = 'xraydata'

In [ ]:
outfile = open(filename,'wb')
pickle.dump(train_data_,outfile)
outfile.close()

In [ ]:
infile = open(filename,'rb')
xraydatalist = pickle.load(infile)
infile.close()

----------------------------------------------------The End-------------------------------------------------------------

Above code is to generate pickle format 

Test Code for testing pickle format

In [ ]:
xraydatalist[0]

In [ ]:
X=[]
Y=[]

for i,j in xraydatalist:
    X.append(i)
    Y.append(j)

X = np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE, 1)
X =X/255.0
Y = np.array(Y).astype(np.float32)
indx = int(0.8*X.shape[0])
trainX, testX = X[:indx,:], X[indx:,:]
trainY, testY = Y[:indx,:], Y[indx:,:]

In [ ]:
testY.shape

In [ ]:
from keras.utils import plot_model
model = Sequential()

model.add(Conv2D(filters = 8, kernel_size = 3, padding = 'same', activation = 'relu', input_shape = trainX.shape[1:]))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.2))

# model.add(Conv2D(filters = 16, kernel_size = 3, padding = 'same', activation = 'relu'))
# model.add(MaxPooling2D(pool_size = 2))
# model.add(Dropout(0.2))
          
# model.add(Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu'))
# model.add(MaxPooling2D(pool_size = 2))
# model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.2))
          
model.add(Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 3))
model.add(Dropout(0.2))

# add in fully connected dense layers to model, then output classifiction probabilities using a sigmoid activation function
model.add(Flatten())
model.add(Dense(500, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(15, activation = 'sigmoid'))


model.compile(loss='binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
model.summary()

In [ ]:
model.fit(X, Y, batch_size=128, epochs=3, validation_split=0.3)

In [ ]:
pred_Y = (model.predict(testX, batch_size = 64, verbose = True)>0.5 ).astype(int)
pred_Y[1]

In [ ]:
from sklearn.metrics import roc_curve,accuracy_score
model.evaluate(testX, testY)

In [ ]:
from sklearn.metrics import roc_curve, auc

# create plot
fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
for (i, label) in enumerate(all_labels):
    fpr, tpr, thresholds = roc_curve(testY[:,i].astype(int), pred_Y[:,i])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (label, auc(fpr, tpr)))

# Set labels for plot
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
fig.savefig('quick_trained_model.png')

In [ ]:
test = []
test = xraydatalist[indx:]
eval= []


for i in range(10):
    eval.append((testY[i], pred_Y[i], test[i][0]))
    
l=0

for i,j,k in eval:
    print("actual:",i," ","predicted:",j)
    plt.imshow(k, cmap='gray')
    plt.show()

In [ ]:
_,acc = model.evaluate(testX, testY,verbose=0)
acc